# Popularity of PUlocationID across green and fhv trip datasets

## Setup
---
Installing of required spark-kafka dependences

---
Import all required libries

In [1]:
import os
os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages org.apache.spark:spark-sql-kafka-0-10_2.12:3.3.1,org.apache.spark:spark-avro_2.12:3.3.1 pyspark-shell'

In [2]:
from pyspark.sql import SparkSession
import pyspark.sql.types as T
import pyspark.sql.functions as F

spark = SparkSession \
    .builder \
    .appName("Spark-Notebook") \
    .getOrCreate()

:: loading settings :: url = jar:file:/usr/local/lib/python3.10/dist-packages/pyspark/jars/ivy-2.5.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /root/.ivy2/cache
The jars for the packages stored in: /root/.ivy2/jars
org.apache.spark#spark-sql-kafka-0-10_2.12 added as a dependency
org.apache.spark#spark-avro_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-5e91458b-cc0f-47bd-ba4f-e793adeacc53;1.0
	confs: [default]
	found org.apache.spark#spark-sql-kafka-0-10_2.12;3.3.1 in central
	found org.apache.spark#spark-token-provider-kafka-0-10_2.12;3.3.1 in central
	found org.apache.kafka#kafka-clients;2.8.1 in central
	found org.lz4#lz4-java;1.8.0 in central
	found org.xerial.snappy#snappy-java;1.1.8.4 in central
	found org.slf4j#slf4j-api;1.7.32 in central
	found org.apache.hadoop#hadoop-client-runtime;3.3.2 in central
	found org.spark-project.spark#unused;1.0.0 in central
	found org.apache.hadoop#hadoop-client-api;3.3.2 in central
	found commons-logging#commons-logging;1.1.3 in central
	found com.google.code.findbugs#jsr305;3.0.0 in central
	found org.apache.common

23/03/15 12:13:33 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


## 1. Reading from Kafka Stream

through `readStream`

#### 1.1 Raw Kafka Stream

In [3]:
# default for startingOffsets is "latest"
df_kafka_raw_green = spark \
    .readStream \
    .format("kafka") \
    .option("kafka.bootstrap.servers", "localhost:9092,broker:29092") \
    .option("subscribe", "green_puid_counts_windowed") \
    .option("startingOffsets", "earliest") \
    .option("checkpointLocation", "checkpoint") \
    .load()

df_kafka_raw_fhv = spark \
    .readStream \
    .format("kafka") \
    .option("kafka.bootstrap.servers", "localhost:9092,broker:29092") \
    .option("subscribe", "fhv_puid_counts_windowed") \
    .option("startingOffsets", "earliest") \
    .option("checkpointLocation", "checkpoint") \
    .load()

In [4]:
df_kafka_raw_green.printSchema(), df_kafka_raw_fhv.printSchema()

root
 |-- key: binary (nullable = true)
 |-- value: binary (nullable = true)
 |-- topic: string (nullable = true)
 |-- partition: integer (nullable = true)
 |-- offset: long (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- timestampType: integer (nullable = true)

root
 |-- key: binary (nullable = true)
 |-- value: binary (nullable = true)
 |-- topic: string (nullable = true)
 |-- partition: integer (nullable = true)
 |-- offset: long (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- timestampType: integer (nullable = true)



(None, None)

#### 1.2 Encoded Kafka Stream

In [5]:
df_kafka_encoded_green = df_kafka_raw_green.selectExpr("CAST(key AS STRING)","CAST(value AS STRING)")

df_kafka_encoded_fhv = df_kafka_raw_fhv.selectExpr("CAST(key AS STRING)","CAST(value AS STRING)")

In [6]:
df_kafka_encoded_green.printSchema(), df_kafka_encoded_fhv.printSchema()

root
 |-- key: string (nullable = true)
 |-- value: string (nullable = true)

root
 |-- key: string (nullable = true)
 |-- value: string (nullable = true)



(None, None)

#### 1.3 Structure Streaming DataFrame

In [7]:
def parse_ride_from_kafka_message(df_raw, schema):
    """ take a Spark Streaming df and parse value col based on <schema>, return streaming df cols in schema """
    assert df_raw.isStreaming is True, "DataFrame doesn't receive streaming data"

    df = df_raw.selectExpr("CAST(key AS STRING)", "CAST(value AS STRING)")

    # split attributes to nested array in one Column
    col = F.split(df['value'], ', ')

    # expand col to multiple top-level columns
    for idx, field in enumerate(schema):
        df = df.withColumn(field.name, col.getItem(idx).cast(field.dataType))
    return df.select([field.name for field in schema])

In [8]:
FHV_SCHEMA = T.StructType(
    [T.StructField("dispatching_base_num", T.StringType()),
     T.StructField('pickup_datetime', T.TimestampType()),
     T.StructField('dropOff_datetime', T.TimestampType()),
     T.StructField("PUlocationID", T.IntegerType()),
     T.StructField("DOlocationID", T.IntegerType()),
     T.StructField("SR_Flag", T.FloatType()),
     T.StructField("Affiliated_base_number", T.StringType()),
     ])




GREEN_SCHEMA = T.StructType(
    [T.StructField("VendorID", T.IntegerType()),
     T.StructField('lpep_pickup_datetime', T.TimestampType()),
     T.StructField('lpep_dropoff_datetime', T.TimestampType()),
     T.StructField("PULocationID", T.IntegerType()),
     T.StructField("DOLocationID", T.IntegerType()),
     T.StructField("passenger_count", T.IntegerType()),
     T.StructField("trip_distance", T.FloatType()),
     ])

In [9]:
df_rides_green = parse_ride_from_kafka_message(df_raw=df_kafka_raw_green, schema=GREEN_SCHEMA)

df_rides_fhv = parse_ride_from_kafka_message(df_raw=df_kafka_raw_fhv, schema=FHV_SCHEMA)

In [10]:
df_rides_green.printSchema(), df_rides_fhv.printSchema()

root
 |-- VendorID: integer (nullable = true)
 |-- lpep_pickup_datetime: timestamp (nullable = true)
 |-- lpep_dropoff_datetime: timestamp (nullable = true)
 |-- PULocationID: integer (nullable = true)
 |-- DOLocationID: integer (nullable = true)
 |-- passenger_count: integer (nullable = true)
 |-- trip_distance: float (nullable = true)

root
 |-- dispatching_base_num: string (nullable = true)
 |-- pickup_datetime: timestamp (nullable = true)
 |-- dropOff_datetime: timestamp (nullable = true)
 |-- PUlocationID: integer (nullable = true)
 |-- DOlocationID: integer (nullable = true)
 |-- SR_Flag: float (nullable = true)
 |-- Affiliated_base_number: string (nullable = true)



(None, None)

## 2 Sink Operation & Streaming Query



#### Console and Memory Sink

In [11]:
def sink_console(df, output_mode: str = 'complete', processing_time: str = '5 seconds'):
    write_query = df.writeStream \
        .outputMode(output_mode) \
        .trigger(processingTime=processing_time) \
        .format("console") \
        .option("truncate", False) \
        .start()
    return write_query # pyspark.sql.streaming.StreamingQuery

In [12]:
# Sink of green trip data
write_query_green = sink_console(df_rides_green.groupby('PULocationID').count())

23/03/15 12:14:07 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-93b509b2-80b5-44d0-8bdb-d605827a51df. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
23/03/15 12:14:07 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.


-------------------------------------------
Batch: 0
-------------------------------------------
+------------+-----+
|PULocationID|count|
+------------+-----+
+------------+-----+

23/03/15 12:14:15 WARN ProcessingTimeExecutor: Current batch is falling behind. The trigger interval is 5000 milliseconds, but spent 7571 milliseconds


In [13]:
# Sink of fhv trip data
write_query_fhv = sink_console(df_rides_fhv.groupby("PULocationID").count())

23/03/15 12:14:20 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-72db1e3c-93e4-4f81-80c7-06d780d22936. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
23/03/15 12:14:20 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.


-------------------------------------------
Batch: 0
-------------------------------------------
+------------+-----+
|PULocationID|count|
+------------+-----+
+------------+-----+



-------------------------------------------
Batch: 1
-------------------------------------------
+------------+-----+
|PULocationID|count|
+------------+-----+
|236         |1    |
|140         |1    |
|null        |139  |
|13          |1    |
|162         |2    |
|261         |1    |
|249         |1    |
|237         |2    |
|141         |1    |
|145         |1    |
+------------+-----+

23/03/15 12:15:51 WARN ProcessingTimeExecutor: Current batch is falling behind. The trigger interval is 5000 milliseconds, but spent 6767 milliseconds


-------------------------------------------
Batch: 2
-------------------------------------------
+------------+-----+
|PULocationID|count|
+------------+-----+
|236         |1    |
|140         |1    |
|null        |582  |
|206         |2    |
|13          |1    |
|16          |2    |
|142         |1    |
|257         |1    |
|88          |1    |
|72          |1    |
|229         |1    |
|173         |4    |
|161         |1    |
|7           |1    |
|162         |2    |
|87          |1    |
|171         |2    |
|129         |1    |
|261         |1    |
|82          |1    |
+------------+-----+
only showing top 20 rows

23/03/15 12:18:36 WARN ProcessingTimeExecutor: Current batch is falling behind. The trigger interval is 5000 milliseconds, but spent 6419 milliseconds


-------------------------------------------
Batch: 3
-------------------------------------------
+------------+-----+
|PULocationID|count|
+------------+-----+
|137         |1    |
|81          |3    |
|236         |3    |
|140         |2    |
|null        |844  |
|206         |2    |
|13          |1    |
|16          |2    |
|142         |2    |
|257         |1    |
|189         |1    |
|88          |1    |
|107         |1    |
|72          |1    |
|229         |1    |
|173         |5    |
|161         |1    |
|7           |1    |
|162         |2    |
|130         |1    |
+------------+-----+
only showing top 20 rows



...

-------------------------------------------
Batch: 105
-------------------------------------------
+------------+-----+
|PULocationID|count|
+------------+-----+
|148         |11100|
|243         |5783 |
|31          |473  |
|137         |6000 |
|85          |2446 |
|251         |537  |
|65          |3350 |
|53          |1189 |
|255         |9583 |
|133         |1631 |
|78          |3348 |
|108         |1052 |
|155         |2126 |
|193         |869  |
|211         |3942 |
|34          |1146 |
|101         |515  |
|115         |432  |
|126         |1980 |
|81          |1983 |
+------------+-----+
only showing top 20 rows

23/03/15 12:32:47 WARN ProcessingTimeExecutor: Current batch is falling behind. The trigger interval is 5000 milliseconds, but spent 8794 milliseconds


-------------------------------------------
Batch: 106
-------------------------------------------
+------------+-----+
|PULocationID|count|
+------------+-----+
|148         |11121|
|243         |5797 |
|31          |476  |
|137         |6023 |
|85          |2448 |
|251         |538  |
|65          |3359 |
|53          |1194 |
|255         |9615 |
|133         |1645 |
|78          |3370 |
|108         |1054 |
|155         |2131 |
|193         |876  |
|211         |3955 |
|34          |1146 |
|101         |518  |
|115         |438  |
|126         |1991 |
|81          |1990 |
+------------+-----+
only showing top 20 rows

23/03/15 12:32:56 WARN ProcessingTimeExecutor: Current batch is falling behind. The trigger interval is 5000 milliseconds, but spent 9522 milliseconds


-------------------------------------------
Batch: 107
-------------------------------------------
+------------+-----+
|PULocationID|count|
+------------+-----+
|148         |11190|
|243         |5840 |
|31          |479  |
|137         |6083 |
|85          |2473 |
|251         |545  |
|65          |3402 |
|53          |1204 |
|255         |9676 |
|133         |1673 |
|78          |3389 |
|108         |1068 |
|155         |2148 |
|193         |885  |
|211         |3982 |
|34          |1150 |
|101         |527  |
|115         |446  |
|126         |2007 |
|81          |2006 |
+------------+-----+
only showing top 20 rows

23/03/15 12:33:04 WARN ProcessingTimeExecutor: Current batch is falling behind. The trigger interval is 5000 milliseconds, but spent 8197 milliseconds


-------------------------------------------
Batch: 108
-------------------------------------------
+------------+-----+
|PULocationID|count|
+------------+-----+
|148         |11237|
|243         |5880 |
|31          |484  |
|137         |6126 |
|85          |2493 |
|251         |545  |
|65          |3428 |
|53          |1215 |
|255         |9698 |
|133         |1701 |
|78          |3422 |
|108         |1081 |
|155         |2168 |
|193         |894  |
|211         |4010 |
|34          |1150 |
|101         |534  |
|115         |449  |
|126         |2021 |
|81          |2026 |
+------------+-----+
only showing top 20 rows

23/03/15 12:33:12 WARN ProcessingTimeExecutor: Current batch is falling behind. The trigger interval is 5000 milliseconds, but spent 7719 milliseconds


-------------------------------------------
Batch: 109
-------------------------------------------
+------------+-----+
|PULocationID|count|
+------------+-----+
|148         |11262|
|243         |5911 |
|31          |488  |
|137         |6146 |
|85          |2502 |
|251         |546  |
|65          |3441 |
|53          |1220 |
|255         |9723 |
|133         |1710 |
|78          |3448 |
|108         |1091 |
|155         |2182 |
|193         |902  |
|211         |4031 |
|34          |1152 |
|101         |539  |
|115         |449  |
|126         |2032 |
|81          |2047 |
+------------+-----+
only showing top 20 rows

23/03/15 12:33:29 WARN ProcessingTimeExecutor: Current batch is falling behind. The trigger interval is 5000 milliseconds, but spent 17033 milliseconds


In [70]:
df_rides_green.join(df_rides_fhv).printSchema()

root
 |-- VendorID: integer (nullable = true)
 |-- lpep_pickup_datetime: timestamp (nullable = true)
 |-- lpep_dropoff_datetime: timestamp (nullable = true)
 |-- PULocationID: integer (nullable = true)
 |-- DOLocationID: integer (nullable = true)
 |-- passenger_count: integer (nullable = true)
 |-- trip_distance: float (nullable = true)
 |-- dispatching_base_num: string (nullable = true)
 |-- pickup_datetime: timestamp (nullable = true)
 |-- dropOff_datetime: timestamp (nullable = true)
 |-- PUlocationID: integer (nullable = true)
 |-- DOlocationID: integer (nullable = true)
 |-- SR_Flag: float (nullable = true)
 |-- Affiliated_base_number: string (nullable = true)



In [86]:
# To join the data if needed

new_df = df_rides_green.join(df_rides_fhv)

In [15]:
write_query_green.stop()
write_query_fhv.stop()